In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/630-final/wn21630project_input_processed.csv')
df.head(1)

In [ ]:
training_size = 800
deving_size = 357
testing_size = 358
size = [training_size, deving_size,testing_size]

In [ ]:
df.fillna(' ',inplace = True)

In [ ]:
raps = []
for index, row in df.iterrows():
    rap = ''
    for i in [ 'verse', 'chorus', 'hook']:
        rap += re.sub('(\n)+', ' . ' , row[i].lower())
    raps.append(rap)

In [ ]:
training = raps[:training_size]
testing = df.intro[testing_size:]

In [ ]:
from collections import *
import gzip
from tqdm.notebook import tqdm

# We'll use NLTK to split, but you could just use simple white-space splitting or regular expressions
try:
    from nltk.tokenize import sent_tokenize, word_tokenize
except:
    def word_tokenize(s):
        return s.split()
    
def safe_word_tokenize(s):
    try:
        return word_tokenize(s)
    except: # Happens without NLTK/punct package installed
        return s.split()

In [ ]:
train = []
for i in training:
    train.append(safe_word_tokenize(i))

In [ ]:
def train_lm(all_data, order=2):
    '''Trains a language model and reutrns it'''
    # "lm" starts for Language Model
    lm = defaultdict(Counter)
    
    for data in tqdm(all_data):
        # This creates a list with the start token "~" repeated for the order of our language model
        pad = ["~"] * order
        data = pad + data + ['[STOP]']

        for i in range(len(data)-order):
            # Context is the words that occur beforehand
            context, word = data[i:i+order], data[i+order]
            # Note: to hash the context, we have to make it a tuple
            lm[tuple(context)][word]+=1

        # Convert the counts into probabilities
        def normalize(counter):
            s = float(sum(counter.values()))
            return [(word,cnt/s) for word,cnt in counter.items()]
 
    outlm = {hist:normalize(words) for hist, words in lm.items()}
    return outlm

In [ ]:
from random import random

# History is a list of words, where we'll only look at the last n=order
def generate_word(lm, history, order):
        history = tuple(history[-order:])
        dist = lm[history]
        x = random()
        for word, v in dist:
            x = x - v
            if x <= 0: return word
def generate_text(lm, order, max_words=500):
    '''Generates a sequence of up to max_words in length'''
    history = ["~"] * order
    out = []
    for i in range(max_words):
        word = generate_word(lm, history, order)
        if word == '[STOP]':
            break
        if order == 0:
            history = tuple()
        else:
            history = history[-order:]
            history.append(word)
        out.append(word)
    return " ".join(out)

In [ ]:
def str_to_tuple(s):
    return tuple(s.split())

In [ ]:
lm = train_lm(train, order=8)

In [ ]:
output_n_gram = []
for i in range(0,358):
    
    output_n_gram.append(generate_text(lm, 8))

In [ ]:
import json
new_dict={'output_song_n_gram':output_n_gram }
with open("./output_song_baseline.json","w") as f:
    json.dump(new_dict,f)
    print("加载入文件完成...")